## Import

In [2]:
import os, sys, email,re
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
import networkx as nx

from nltk.tokenize.regexp import RegexpTokenizer

from subprocess import check_output

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# from sklearn.lda import LDA

from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer

## The Data

In [220]:
tracks_df = pd.read_csv('raw_tracks.csv')
tracks_df.head()

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...


In [169]:
tracks_df1 = pd.read_csv('tracks.csv')
# tracks_df.track_genres

In [156]:
analysis_df=tracks_df[['track_id', 'album_title', 'track_title']]
analysis_df.head()

,track_id,album_title,track_title
0,2,AWOL - A Way Of Life,Food
1,3,AWOL - A Way Of Life,Electric Ave
2,5,AWOL - A Way Of Life,This World
3,10,Constant Hitmaker,Freeway
4,20,Niris,Spiritual Level


## Clean and Lemmatize

In [53]:
def clean(text):
    stop = set(stopwords.words('english'))
#     stop.update(("Generic","words"))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
#     porter= PorterStemmer()
    
    text=str(text).rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return normalized

In [54]:
text_clean=[]
for text in analysis_df['track_title']:
    text_clean.append(clean(text).split())

## The Model

In [57]:
dictionary = corpora.Dictionary(text_clean)
text_term_matrix = [dictionary.doc2bow(text) for text in text_clean]

In [58]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(text_term_matrix, num_topics=5, id2word = dictionary, passes=20)

In [59]:
top_words = ldamodel.print_topics(num_topics=5, num_words=20)

In [152]:
topics_df = []
for i in range(0,5):
    new_line = re.sub(r'[0-9]+', '',top_words[i][1])
    new_line = new_line.replace('*','')
    new_line = new_line.replace('"','')
    new_line = new_line.replace('.','')
    new_line = new_line.strip().split('+')
    topics_df.append(new_line)

In [171]:
pd.DataFrame(topics_df) #TopWords

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,de,la,part,love,instrumental,live,le,l,day,e,c,black,sun,man,dance,death,home,city,improv,du
1,remix,b,untitled,go,come,back,featuring,get,moon,intro,sky,street,face,music,red,water,soul,god,die,see
2,feat,light,version,like,way,r,dark,u,dead,v,let,el,space,old,summer,thing,free,p,rain,first
3,time,mix,id,dream,blue,n,new,life,world,electric,lost,ft,girl,theme,never,take,away,happy,sea,et
4,one,song,night,ii,track,heart,side,eye,last,star,good,en,mind,un,know,little,want,hand,bad,say


In [138]:
prob_output=[]
for text in text_clean:
    prob_output.append(list(pd.DataFrame(ldamodel[dictionary.doc2bow(text)])[1]))

In [142]:
# list(pd.DataFrame(ldamodel[dictionary.doc2bow(["text"])])[1])

In [130]:
# text_clean[1],ldamodel[dictionary.doc2bow(text_clean[1])]

(['electric', 'ave'],
 [(0, 0.066667321053759582),
  (1, 0.066667335159745725),
  (2, 0.066667317366228093),
  (3, 0.40008447182406481),
  (4, 0.39991355459620181)])

## Probabilities as Features

In [140]:
df_test= pd.DataFrame(prob_output, columns=["prob_cluster_1","prob_cluster_2","prob_cluster_3","prob_cluster_4","prob_cluster_5"])

In [172]:
df_test.head()

,prob_cluster_1,prob_cluster_2,prob_cluster_3,prob_cluster_4,prob_cluster_5
0,0.100001,0.100001,0.100001,0.599996,0.100001
1,0.066667,0.066667,0.066667,0.400084,0.399914
2,0.100000,0.100000,0.100000,0.600000,0.100000
3,0.100003,0.100004,0.100004,0.100003,0.599986
4,0.733331,0.066667,0.066667,0.066667,0.066667


In [221]:
tracks_probability = pd.concat([tracks_df[['artist_name','track_id','track_title']], df_test], axis=1)

In [222]:
tracks_probability.to_csv('tracks_probability.csv')

In [173]:
df_original= pd.read_csv('df_final.csv')

In [174]:
df_original.head()

,track_id,track_title,track_genre_top,track_genres,track_genres_all,chroma_cens_kurtosis,chroma_cens.1_kurtosis,chroma_cens.2_kurtosis,chroma_cens.3_kurtosis,chroma_cens.4_kurtosis,...,tonnetz.39_std,tonnetz.40_std,tonnetz.41_std,zcr_kurtosis,zcr.1_max,zcr.2_mean,zcr.3_median,zcr.4_min,zcr.5_skew,zcr.6_std
0,2,Food,Hip-Hop,[21],[21],7.180653,5.230309,0.249321,1.347620,1.482478,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,Electric Ave,Hip-Hop,[21],[21],1.888963,0.760539,0.345297,2.295201,1.654031,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,This World,Hip-Hop,[21],[21],0.527563,-0.077654,-0.279610,0.685883,1.937570,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,Freeway,Pop,[10],[10],3.702245,-0.291193,2.196742,-0.234449,1.367364,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,Spiritual Level,NaN,"[76, 103]","[17, 10, 76, 103]",-0.193837,-0.198527,0.201546,0.258556,0.775204,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [223]:
df=tracks_probability.merge(df_original[['track_id','track_genre_top']], on='track_id', how='left')

In [224]:
df.head()

,artist_name,track_id,track_title,prob_cluster_1,prob_cluster_2,prob_cluster_3,prob_cluster_4,prob_cluster_5,track_genre_top
0,AWOL,2,Food,0.100001,0.100001,0.100001,0.599996,0.100001,Hip-Hop
1,AWOL,3,Electric Ave,0.066667,0.066667,0.066667,0.400084,0.399914,Hip-Hop
2,AWOL,5,This World,0.100000,0.100000,0.100000,0.600000,0.100000,Hip-Hop
3,Kurt Vile,10,Freeway,0.100003,0.100004,0.100004,0.100003,0.599986,Pop
4,Nicky Cook,20,Spiritual Level,0.733331,0.066667,0.066667,0.066667,0.066667,NaN


In [225]:
df['max_prob']= df[['prob_cluster_1','prob_cluster_2','prob_cluster_3','prob_cluster_4','prob_cluster_5']].idxmax(axis=1)

In [226]:
df.head()

,artist_name,track_id,track_title,prob_cluster_1,prob_cluster_2,prob_cluster_3,prob_cluster_4,prob_cluster_5,track_genre_top,max_prob
0,AWOL,2,Food,0.100001,0.100001,0.100001,0.599996,0.100001,Hip-Hop,prob_cluster_4
1,AWOL,3,Electric Ave,0.066667,0.066667,0.066667,0.400084,0.399914,Hip-Hop,prob_cluster_4
2,AWOL,5,This World,0.100000,0.100000,0.100000,0.600000,0.100000,Hip-Hop,prob_cluster_4
3,Kurt Vile,10,Freeway,0.100003,0.100004,0.100004,0.100003,0.599986,Pop,prob_cluster_5
4,Nicky Cook,20,Spiritual Level,0.733331,0.066667,0.066667,0.066667,0.066667,NaN,prob_cluster_1


In [209]:
df_an=df.groupby(['max_prob', 'track_genre_top'])['track_id'].count()

In [210]:
df_an.to_csv('count_new.csv')

In [214]:
from PyLyrics import *

In [228]:
PyLyrics.getLyrics('Nicky Cook','Spiritual Level')

ValueError: Song or Singer does not exist or the API does not have Lyrics